<h1 style='font-size: 35px; color: Tomato; font-family: Colonna MT; font-weight: 700; text-align: center; margin-bottom: 10px;'>Compact Letter Displays (Statistically Significant Letters)</h1>

---

<h1 style='font-size: 20px; color: crimson; font-family: Candara; font-weight: 600'>1.0. Import Required Libraries</h1>

In [16]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols as smf_ols 
from statsmodels.stats.anova import anova_lm 
import pandas as pd
import numpy as np
import string
import re

<h1 style='font-size: 20px; color: crimson; font-family: Candara; font-weight: 600'>2.0. Compact Letter Display (CLD)</h1>

Below is a function and accompanying scripts that will **automatically compute ANOVA and Tukey’s HSD** for all selected **metrics of interest** in the dataset. The function iterates through each metric, performs **ANOVA** to check for significant differences, then applies **Tukey’s HSD test** to determine which groups differ. Finally, it compiles all results into a **structured format**, ensuring a clear and organized presentation of statistical outcomes for easy interpretation.

In [9]:
def rename(text):
    text = re.sub(r'[^a-zA-Z]', "",  text) 
    return text

def Compact_letters_display(df, Factor='', exclude_cols=None):
    if exclude_cols is None:
        exclude_cols = []

    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
    def calculate_stats_for_column(col_name):
        model = smf_ols(f'{col_name}~C({Factor})', data=df).fit()
        anova_results = anova_lm(model)
        p_value = anova_results.loc[f'C({Factor})', 'PR(>F)']
        tukey = pairwise_tukeyhsd(df[col_name], df[Factor])
        Turkey_results = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
        group_labels = letters(Turkey_results)
        stats = df.groupby(Factor)[col_name].agg(['mean', 'sem']).round(2)
        stats['letter'] = stats.index.map(group_labels)
        stats['formatted'] = stats.apply(
            lambda x: f"{x['mean']:.2f} ± {x['sem']:.2f} {x['letter']}", axis=1)
        
        return stats['formatted'], p_value
    results = {}
    p_values = {}
    for col in numeric_cols:
        results[col], p_values[col] = calculate_stats_for_column(col)
    
    results_df = pd.DataFrame(results)
    results_df.index.name = Factor
    p_values_formatted = {col: f"{p:.4f}" + ('***' if p <= 0.001 else '**' if p <= 0.01 else '*' if p <= 0.05 else 'ns') 
                         for col, p in p_values.items()}
    p_value_df = pd.DataFrame([p_values_formatted], index=['p-value'])
    final_results = pd.concat([results_df, p_value_df])
    return final_results

def letters(df, alpha=0.05):
    df["p-adj"] = df["p-adj"].astype(float)
    group1 = set(df.group1.tolist())
    group2 = set(df.group2.tolist())
    groupSet = group1 | group2
    groups = list(groupSet)
    letters = list(string.ascii_lowercase)[:len(groups)]
    cldgroups = letters
    cld = pd.DataFrame(list(zip(groups, letters, cldgroups)))
    cld[3] = ""
    
    for row in df.itertuples():
        if df["p-adj"][row[0]] > alpha:
            cld.loc[groups.index(df["group1"][row[0]]), 2] += cld.loc[groups.index(df["group2"][row[0]]), 1]
            cld.loc[groups.index(df["group2"][row[0]]), 2] += cld.loc[groups.index(df["group1"][row[0]]), 1]
        if df["p-adj"][row[0]] < alpha:
            cld.loc[groups.index(df["group1"][row[0]]), 3] += cld.loc[groups.index(df["group2"][row[0]]), 1]
            cld.loc[groups.index(df["group2"][row[0]]), 3] += cld.loc[groups.index(df["group1"][row[0]]), 1]
    
    cld[2] = cld[2].apply(lambda x: "".join(sorted(x)))
    cld[3] = cld[3].apply(lambda x: "".join(sorted(x)))
    cld.rename(columns={0: "groups"}, inplace=True)
    cld = cld.sort_values(cld.columns[2], key=lambda x: x.str.len())
    cld["labels"] = ""
    letters = list(string.ascii_lowercase)
    unique = []
    
    for item in cld[2]:
        for fitem in cld["labels"].unique():
            for c in range(len(fitem)):
                if not set(unique).issuperset(set(fitem[c])):
                    unique.append(fitem[c])
        g = len(unique)
        for kitem in cld[1]:
            if kitem in item:
                if cld.loc[cld[1] == kitem, "labels"].iloc[0] == "":
                    cld.loc[cld[1] == kitem, "labels"] += letters[g]
                if kitem in " ".join(cld.loc[cld["labels"] == letters[g], 3]):
                    g = len(unique) + 1
                if len(set(cld.loc[cld[1] == kitem, "labels"].iloc[0]).intersection(cld.loc[cld[2] == item, "labels"].iloc[0])) <= 0:
                    if letters[g] not in list(cld.loc[cld[1] == kitem, "labels"].iloc[0]):
                        cld.loc[cld[1] == kitem, "labels"] += letters[g]
                    if letters[g] not in list(cld.loc[cld[2] == item, "labels"].iloc[0]):
                        cld.loc[cld[2] == item, "labels"] += letters[g]
    
    cld = cld.sort_values("labels")
    cld.drop(columns=[1, 2, 3], inplace=True)
    cld = dict(zip(cld["groups"], cld["labels"]))
    return cld

def main(data, group, savepath=None):
    data.columns = data.columns.to_series().apply(rename)
    results_df = Compact_letters_display(data, Factor=group)
    results_df_columns = df.select_dtypes(include=[np.number]).columns
    results_df.columns = results_df_columns
    if savepath: results_df.to_csv(savepath, index=False)
    return results_df

<h1 style='font-size: 20px; color: crimson; font-family: Candara; font-weight: 600'>3.0. Import and Preprocessing Dataset</h1>

In [10]:
filepath = "Datasets/Planetary Datasets.xlsx"
df = pd.read_excel(filepath)
df.head(10)

,PlanetType,OrbitZone,Orbital Velocity (km/s),Atmospheric Density (kg/m³),Surface Temperature (⁰C),Root Length (cm),Gravitational Accerelation (m/s²),Planet Radius (km),Solar Exposure (W/m²),MagField Strength (μT)
0,Gas Titans,Inner,5856.15,1.86,46.52,24.32,11.99,1953.59,6.69,242.41
1,Ice Giants,Habitable,4669.68,1.39,34.69,17.66,8.67,1536.62,6.17,233.66
2,Gas Titans,Outer,5832.55,2.04,40.82,27.00,9.50,1638.99,5.41,230.07
3,Gas Titans,Habitable,4273.44,1.40,29.07,19.59,10.35,1244.93,4.26,154.25
4,Ice Giants,Habitable,4182.42,1.30,34.73,19.78,10.55,1514.08,4.64,200.54
5,Ice Giants,Inner,8638.83,2.81,63.05,39.40,17.06,2322.17,8.10,319.83
6,Gas Titans,Habitable,4021.96,1.64,36.07,18.14,9.25,1405.11,4.21,195.13
7,Terestrial,Inner,8169.40,2.58,59.84,28.51,13.84,1941.60,8.73,291.64
8,Gas Titans,Habitable,4086.26,1.14,38.28,20.74,11.20,1488.94,5.14,171.97
9,Gas Titans,Outer,6670.09,1.65,37.32,24.34,11.70,1814.38,5.45,187.35


<h1 style='font-size: 18px; color: crimson; font-family: Candara; font-weight: 600'>4.0: Comparing the Means of Variables Across Different Groups</h1>

*Now, let’s turn our attention to comparing the means of variables across different specified groups. This approach helps us understand how each variable behaves within various categories or groups. For instance, we might explore how the average outcome of a variable changes across different practices or conditions. Such comparisons allow us to identify any significant differences between groups, uncovering patterns or trends that could be crucial for deeper analysis. By analyzing these mean comparisons, we gain valuable insights into the relationships between variables and groups, which can inform decisions or guide further investigations.*

In [17]:
def summary_stats(df, Metrics, group=''):
    df_1 = df.drop(columns=[group], errors='ignore')
    grand_mean = df_1[Metrics].mean().round(2)
    sem = df_1[Metrics].sem().round(2)
    cv = (df_1[Metrics].std() / df_1[Metrics].mean() * 100).round(2)
    grouped = df.groupby(group)[Metrics].agg(['mean', 'sem']).reset_index()
    summary_df = pd.DataFrame()
    for col in Metrics:
        summary_df[col] = grouped.apply(
            lambda x: f"{x[(col, 'mean')]:.2f} ± {x[(col, 'sem')]:.2f}", axis=1
        )
    
    summary_df.insert(0, group, grouped[group])
    grand_mean_row = ['Grand Mean'] + grand_mean.tolist()
    sem_row = ['SEM'] + sem.tolist()
    cv_row = ['%CV'] + cv.tolist()
    
    summary_df.loc[len(summary_df)] = grand_mean_row
    summary_df.loc[len(summary_df)] = sem_row
    summary_df.loc[len(summary_df)] = cv_row
    
    return summary_df

Metrics = df.select_dtypes(include=np.number).columns.tolist()
Results = summary_stats(df, Metrics, group='PlanetType')
Results.T

,0,1,2,3,4,5
PlanetType,Gas Titans,Ice Giants,Terestrial,Grand Mean,SEM,%CV
Orbital Velocity (km/s),5455.29 ± 157.64,6561.31 ± 269.47,6183.59 ± 242.16,6057.51,136.29,24.65
Atmospheric Density (kg/m³),1.68 ± 0.04,1.94 ± 0.08,1.84 ± 0.08,1.82,0.04,25.24
Surface Temperature (⁰C),39.76 ± 1.12,44.85 ± 1.77,41.60 ± 1.67,42.03,0.9,23.47
Root Length (cm),21.97 ± 0.58,25.15 ± 1.03,24.53 ± 0.86,23.86,0.5,22.78
Gravitational Accerelation (m/s²),10.91 ± 0.26,12.56 ± 0.47,12.48 ± 0.56,11.97,0.26,24.11
Planet Radius (km),1712.77 ± 48.24,1936.37 ± 81.52,1811.51 ± 74.51,1818.36,40.42,24.35
Solar Exposure (W/m²),5.68 ± 0.17,6.41 ± 0.26,6.43 ± 0.25,6.17,0.14,24.13
MagField Strength (μT),221.38 ± 6.67,256.74 ± 9.06,247.81 ± 9.74,241.68,5.09,23.06


<h1 style='font-size: 20px; color: crimson; font-family: Candara; font-weight: 600'>5.0. Compact Letter Display (CLD) implementations</h1>


In [13]:
data = df.copy()
results = main(data, group="PlanetType", savepath=None)
display(results.T)

,Gas Titans,Ice Giants,Terestrial,p-value
Orbital Velocity (km/s),5455.29 ± 157.64 a,6561.31 ± 269.47 b,6183.59 ± 242.16 ab,0.0028**
Atmospheric Density (kg/m³),1.68 ± 0.04 a,1.94 ± 0.08 b,1.84 ± 0.08 ab,0.0313*
Surface Temperature (⁰C),39.76 ± 1.12 a,44.85 ± 1.77 a,41.60 ± 1.67 a,0.0646ns
Root Length (cm),21.97 ± 0.58 a,25.15 ± 1.03 b,24.53 ± 0.86 ab,0.0192*
Gravitational Accerelation (m/s²),10.91 ± 0.26 a,12.56 ± 0.47 b,12.48 ± 0.56 b,0.0138*
Planet Radius (km),1712.77 ± 48.24 a,1936.37 ± 81.52 a,1811.51 ± 74.51 a,0.0766ns
Solar Exposure (W/m²),5.68 ± 0.17 a,6.41 ± 0.26 a,6.43 ± 0.25 a,0.0340*
MagField Strength (μT),221.38 ± 6.67 a,256.74 ± 9.06 b,247.81 ± 9.74 ab,0.0113*


In [15]:
data = df.copy()
results = main(data, group="OrbitZone", savepath=None)
display(results.T)

,Habitable,Inner,Outer,p-value
Orbital Velocity (km/s),4539.29 ± 77.92 c,7438.93 ± 180.66 b,6378.21 ± 130.30 a,0.0000***
Atmospheric Density (kg/m³),1.37 ± 0.02 c,2.27 ± 0.06 b,1.87 ± 0.03 a,0.0000***
Surface Temperature (⁰C),33.02 ± 0.62 c,52.17 ± 1.18 b,41.76 ± 0.88 a,0.0000***
Root Length (cm),18.53 ± 0.26 c,28.84 ± 0.78 b,24.84 ± 0.37 a,0.0000***
Gravitational Accerelation (m/s²),9.29 ± 0.16 c,14.77 ± 0.38 b,12.12 ± 0.27 a,0.0000***
Planet Radius (km),1385.84 ± 20.92 c,2275.62 ± 55.30 b,1838.92 ± 29.79 a,0.0000***
Solar Exposure (W/m²),4.68 ± 0.07 c,7.66 ± 0.18 b,6.32 ± 0.11 a,0.0000***
MagField Strength (μT),187.67 ± 3.29 c,294.33 ± 7.03 b,249.19 ± 4.95 a,0.0000***


---

This analysis was performed by **Jabulente**, a passionate and dedicated data scientist with a strong commitment to using data to drive meaningful insights and solutions. For inquiries, collaborations, or further discussions, please feel free to reach out via.  

---

<div align="center">  
    
[![GitHub](https://img.shields.io/badge/GitHub-Jabulente-black?logo=github)](https://github.com/Jabulente)  [![LinkedIn](https://img.shields.io/badge/LinkedIn-Jabulente-blue?logo=linkedin)](https://linkedin.com/in/jabulente-208019349)  [![X (Twitter)](https://img.shields.io/badge/X-@Jabulente-black?logo=x)](https://x.com/Jabulente)  [![Instagram](https://img.shields.io/badge/Instagram-@Jabulente-purple?logo=instagram)](https://instagram.com/Jabulente)  [![Threads](https://img.shields.io/badge/Threads-@Jabulente-black?logo=threads)](https://threads.net/@Jabulente)  [![TikTok](https://img.shields.io/badge/TikTok-@Jabulente-teal?logo=tiktok)](https://tiktok.com/@Jabulente)  [![Email](https://img.shields.io/badge/Email-jabulente@hotmail.com-red?logo=gmail)](mailto:Jabulente@hotmail.com)  

</div>

</div>

<h5 style='font-size: 65px; color: crimson; font-family: Colonna MT; font-weight: 600; text-align: center'>THE END</h5>